### Natural Language Processing

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
print(spacy.__version__)

2.3.5


In [3]:
doc = nlp("Tea is healthy and calming, don't you think?")

#### Tokenizing

In [4]:
for token in doc:
    print(token)

Tea
is
healthy
and
calming
,
do
n't
you
think
?


In [5]:
# lemmatize the words
print(f"Token \t\tLemma \t\tStopword".format('Token', 'Lemma',
                                             'Stopword'))
print("-"*40)
for token in doc:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}")

Token 		Lemma 		Stopword
----------------------------------------
Tea		tea		False
is		be		True
healthy		healthy		False
and		and		True
calming		calm		False
,		,		False
do		do		True
n't		not		True
you		-PRON-		True
think		think		False
?		?		False


In [6]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
# print(matcher)

In [7]:
terms = ['Galaxy Note', 'iPhone 11', 'iPhone XS', 'Google Pixel']
patterns = [nlp(text) for text in terms]
matcher.add("TerminologyList", patterns)
matcher

In [8]:
text_doc = nlp("Glowing review overall, and some really interesting side-by-side "
               "photography tests pitting the iPhone 11 Pro against the "
               "Galaxy Note 10 Plus and last year’s iPhone XS and Google Pixel 3.")

In [9]:
matches = matcher(text_doc)
print(matches)

[(3766102292120407359, 17, 19), (3766102292120407359, 22, 24), (3766102292120407359, 30, 32), (3766102292120407359, 33, 35)]


In [10]:
match_id, start, end = matches[0]
print(nlp.vocab.strings[match_id], text_doc[start:end])

TerminologyList iPhone 11


In [11]:
for match in matches:
   print(f"Token number {match[1]}: {text_doc[start:end]}")

Token number 17: iPhone 11
Token number 22: iPhone 11
Token number 30: iPhone 11
Token number 33: iPhone 11


New Exercise

In [12]:
import pandas as pd
data = pd.read_json('resturant.json')
data.head()


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


In [13]:
# List of menu items and common alternative spellings
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",
         "Prosciutto", "Salami"]


#### Plan the Analysis
You could group reviews by what menu items they mention,
and then calculate the average rating for reviews that mentioned each item.
You can tell which foods are mentioned in reviews with low scores,
so the restaurant can fix the recipe or remove those foods from the menu.

In [14]:
# Creating for one review
import spacy
from spacy.matcher import PhraseMatcher

index_of_review_to_test_on = 14
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

# Load the SpaCy model
nlp = spacy.blank('en')

# Create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)

# Create the PhraseMatcher object. The tokenizer is the first argument. Use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
# print(matcher)
# Create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

# Add the item patterns to the matcher.
# Look at https://spacy.io/api/phrasematcher#add in the docs for help with this step
# Then uncomment the lines below

#
matcher.add("MENU",            # Just a name for the set of rules we're matching to
           menu_tokens_list
          )

# Find matches in the review_doc
matches = matcher(review_doc)
print(matches)

[(8291075388056826051, 2, 3), (8291075388056826051, 16, 17), (8291075388056826051, 58, 59)]


In [15]:
print(review_doc)
for match in matches:
   print(f"Token number {match[1]}: {review_doc[match[1]:match[2]]}")

The Il Purista sandwich has become a staple of my life. Mozzarella, basil, prosciutto, roasted red peppers and balsamic vinaigrette blend into a front runner for the best sandwich in the valley. Goes great with sparkling water or a beer. 

DeFalco's also has other Italian fare such as a delicious meatball sub and classic pastas.
Token number 2: Purista
Token number 16: prosciutto
Token number 58: meatball


Matching on Whole Dataset

In [16]:
from collections import defaultdict

# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review.text)
    # Using the matcher from the previous exercise
    matches = matcher(doc)

    # Create a set of the items found in the review text
    found_items = set([doc[match[1]:match[2]].lower_ for match in matches])

    # Update item_ratings with rating for each item in found_items
    # Transform the item strings to lowercase to make it case insensitive
    for item in found_items:
        item_ratings[item].append(review.stars)

In [18]:
# item_ratings


In [20]:
# Calculate the mean ratings for each menu item as a dictionary
mean_ratings = {item:sum(ratings)/len(ratings) for item, ratings in item_ratings.items()}

# Find the worst item, and write it as a string in worst_item. This can be multiple lines of code if you want.
worst_item = sorted(mean_ratings, key=mean_ratings.get)[0]


In [22]:
print("items with worst ratigs {}".format(worst_item))
print("items with mean ratings {}".format(mean_ratings[worst_item]))

items with worst ratigs chicken cutlet
items with mean ratings 3.4


In [23]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}
item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

                    pizza  265
                    pasta  206
                 meatball  128
              cheesesteak   97
             cheese steak   76
                  cannoli   72
                  calzone   72
                 eggplant   69
                  purista   63
                  lasagna   59
          italian sausage   53
               prosciutto   50
             chicken parm   50
             garlic bread   39
                  gnocchi   37
                spaghetti   36
                 calzones   35
                   pizzas   32
                   salami   28
            chicken pesto   27
             italian beef   25
            italian combo   21
                 tiramisu   21
                     ziti   21
         chicken parmesan   19
       chicken parmigiana   17
               portobello   14
           mac and cheese   11
           chicken cutlet   10
         steak and cheese    9
                 pastrami    9
               roast beef    7
       f

In [24]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Worst rated menu items:
chicken cutlet       Ave rating: 3.40 	count: 10
turkey sandwich      Ave rating: 3.80 	count: 5
spaghetti            Ave rating: 3.89 	count: 36
italian beef         Ave rating: 3.92 	count: 25
tuna salad           Ave rating: 4.00 	count: 5
macaroni             Ave rating: 4.00 	count: 5
italian combo        Ave rating: 4.05 	count: 21
garlic bread         Ave rating: 4.13 	count: 39
roast beef           Ave rating: 4.14 	count: 7
eggplant             Ave rating: 4.16 	count: 69


Best rated menu items:
chicken pesto        Ave rating: 4.56 	count: 27
chicken salad        Ave rating: 4.60 	count: 5
purista              Ave rating: 4.67 	count: 63
prosciutto           Ave rating: 4.68 	count: 50
reuben               Ave rating: 4.75 	count: 4
steak and cheese     Ave rating: 4.89 	count: 9
artichoke salad      Ave rating: 5.00 	count: 5
fettuccini alfredo   Ave rating: 5.00 	count: 6
turkey breast        Ave rating: 5.00 	count: 1
corned beef          Ave ratin